In [2]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras

model = keras.models.load_model('persian_digits.h5')

# Read the input image (make sure it's a binary image)
image = cv2.imread('images/persian-digits.jpg', cv2.IMREAD_GRAYSCALE)
image_annotated = cv2.imread('images/persian-digits.jpg')

# Apply thresholding to create a binary image
_, binary_image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)
binary_image = 255-binary_image

# Use connectedComponentsWithStats to obtain labels and bounding boxes
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary_image, connectivity=8)

# Iterate through each connected component
for label in range(1, num_labels):
    # Get the bounding box of the current connected component
    x, y, w, h = stats[label][:4]

    if h>5 and w>5:
        # Crop the connected component using the bounding box
        cropped_img = binary_image[y:y+h, x:x+w]
        resized_img = cv2.resize(cropped_img, (10,10))
        reshaped_img = resized_img.reshape(1, 100)
        x_ = reshaped_img.astype('float32')/255 #normalization
        p = model.predict(x_)
        result= np.argmax(p)
        print(result)
        y = y-10 if y>10 else y+10
        cv2.putText(image_annotated, str(result), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255))

# Display or save the cropped connected component
cv2.imshow('result image', image_annotated)
cv2.waitKey(0)
cv2.destroyAllWindows()

1/1 [==============================] - 0s 210ms/step
2
1/1 [==============================] - 0s 36ms/step
3
1/1 [==============================] - 0s 33ms/step
6
1/1 [==============================] - 0s 31ms/step
7


In [3]:
result

7

In [4]:
np.argmax(result)

0

In [5]:
stats

array([[     0,      0,    700,    525, 365699],
       [   181,    215,     38,     71,    447],
       [   386,    227,     48,     55,    475],
       [   531,    307,     28,     57,    435],
       [   379,    308,      1,      2,      2],
       [   262,    349,     47,     43,    439],
       [   344,    377,      1,      1,      1],
       [   343,    380,      2,      1,      2]], dtype=int32)

In [8]:
cap = cv2.VideoCapture(0)
model = keras.models.load_model('persian_digits.h5')

while True:
    ret, color_frame = cap.read()
    gray_frame = cv2.cvtColor(color_frame, cv2.COLOR_BGR2GRAY)
    cv2.GaussianBlur(gray_frame, (7,7), 0)
    # Apply thresholding to create a binary image
    _, binary_image = cv2.threshold(gray_frame, 128, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    binary_image = 255-binary_image
    # Use connectedComponentsWithStats to obtain labels and bounding boxes
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(binary_image, connectivity=8)
        # Iterate through each connected component
    for label in range(1, num_labels):
        # Get the bounding box of the current connected component
        x, y, w, h = stats[label][:4]
    
        if h>5 and w>5:
            # Crop the connected component using the bounding box
            cropped_img = binary_image[y:y+h, x:x+w]
            resized_img = cv2.resize(cropped_img, (10,10))
            _, bimage = cv2.threshold(resized_img, 128, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            #dilation = cv2.dilate(bimage, np.ones((5,5),np.uint8), iterations =1)
            closing = cv2.morphologyEx(bimage, cv2.MORPH_CLOSE, np.ones((5,5),np.uint8))
            #reshaped_img = dilation.reshape(1, 100)
            reshaped_img = closing.reshape(1, 100)
            x_ = reshaped_img.astype('float32')/255 #normalization
            p = model.predict(x_)
            if np.max(p)<0.6:
                continue
            result= np.argmax(p)
            print(result)
            y = y-10 if y>10 else y+10
            cv2.putText(color_frame, str(result), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255))

    cv2.imshow('Webcam', color_frame)
    cv2.imshow('Webcam binary', binary_image)
    if cv2.waitKey(1) & 0xFF == 27:
        break
        
# Release camera and close windows
cap.release()
cv2.destroyAllWindows()  

1/1 [==============================] - 0s 34ms/step
1
1/1 [==============================] - 0s 32ms/step
1
1/1 [==============================] - 0s 30ms/step
6
1/1 [==============================] - 0s 32ms/step
5
1/1 [==============================] - 0s 32ms/step
1
1/1 [==============================] - 0s 33ms/step
5
1/1 [==============================] - 0s 32ms/step
1
1/1 [==============================] - 0s 30ms/step
1
1/1 [==============================] - 0s 33ms/step
1
1/1 [==============================] - 0s 33ms/step
1
1/1 [==============================] - 0s 32ms/step
1
1/1 [==============================] - 0s 34ms/step
1
1/1 [==============================] - 0s 31ms/step
6
1/1 [==============================] - 0s 31ms/step
1
1/1 [==============================] - 0s 30ms/step
1
1/1 [==============================] - 0s 32ms/step
1
1/1 [==============================] - 0s 31ms/step
1
1/1 [==============================] - 0s 30ms/step
1
1/1 [=======================

KeyboardInterrupt: 

In [9]:
cap.release()
cv2.destroyAllWindows()